In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from bs4 import BeautifulSoup
import requests, re

In [57]:
df = pd.read_excel('data_files/감성사전_편집.xlsx')

In [58]:
df

,Unnamed: 0,슬픔,분노,기쁨,긴장감,무서운,증오,차분히,섹시,브릿락,설렘,몽환
0,가련하다,1,0,0,0,0,0,0,0,0,0,0
1,가슴 아프다,1,0,0,0,0,0,0,0,0,0,0
2,가슴앓이,1,0,0,0,0,0,0,0,0,0,0
3,가엾다,1,0,0,0,0,0,0,0,0,0,0
4,각박하다,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3999,한계였다,1,0,0,0,0,0,0,0,0,0,0
4000,멸망했다,1,0,0,0,0,0,0,0,0,0,0
4001,씁쓸한,1,0,0,0,0,0,0,0,0,0,0
4002,급히,0,0,0,1,0,0,0,0,0,0,0


In [59]:
df.set_index('Unnamed: 0', drop=True, inplace=True)

In [140]:
url = 'https://novel.naver.com/webnovel/detail?novelId=920928&volumeNo=1'

In [141]:
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

In [142]:
text = soup.select('div.detail_view_content')[0].text
text = ' '.join(re.compile('[ㄱ-ㅎ가-힣]+').findall(text))

In [143]:
from konlpy.tag import Okt, Mecab
okt = Okt()
mecab = Mecab('C:/mecab/mecab-ko-dic/')

In [144]:
okt_li = okt.morphs(text)
mecab_li = mecab.morphs(text)

In [145]:
word_li = [i for i in okt_li if len(i) >= 2]

In [146]:
std_df = pd.read_csv('data_files/stopwords-ko.txt')

In [147]:
word_li = [i for i in word_li if i not in std_df ]

In [148]:
mood_count = {'슬픔': 0,
'분노': 0,
'기쁨': 0,
'긴장감': 0,
'무서운': 0,
'증오': 0,
'차분히': 0,
'섹시': 0,
'브릿락': 0,
'설렘': 0,
'몽환': 0}
mood_words = {}

for word in word_li:
    if word in df.index:
        word_mood = df.columns[df.loc[word, :] == 1].values[0]
        if word_mood in mood_words:
            mood_count[word_mood] += 1
            mood_words[word_mood].append(word)
        else:
            mood_count[word_mood] += 1
            mood_words[word_mood] =[word]
for mood in mood_words.keys():
    mood_words[mood] = set(mood_words[mood])

In [149]:
mood_count

{'슬픔': 18,
 '분노': 5,
 '기쁨': 16,
 '긴장감': 24,
 '무서운': 1,
 '증오': 3,
 '차분히': 3,
 '섹시': 9,
 '브릿락': 3,
 '설렘': 4,
 '몽환': 8}

In [150]:
mood_count.values()

dict_values([18, 5, 16, 24, 1, 3, 3, 9, 3, 4, 8])

In [151]:
mood_words

{'몽환': {'신경', '심장'},
 '슬픔': {'가난한',
  '관계',
  '광대',
  '기었고',
  '꿇어',
  '뒤통수',
  '불구',
  '아이고',
  '애원',
  '없다',
  '절규',
  '좌절',
  '포기',
  '한계였다'},
 '기쁨': {'멋진', '바르고', '보람', '성공', '웃었다', '이익', '절친', '좋은', '최고', '축하'},
 '설렘': {'기대', '미소'},
 '긴장감': {'날카롭게', '눈치', '독종', '떨렸다', '물어', '밟아', '사냥개', '야심', '중요한'},
 '섹시': {'가슴', '관심', '높여', '눈빛', '미친', '이해', '인정'},
 '분노': {'고집', '악다물었다', '죽이는'},
 '증오': {'빈정거리는', '조롱', '한심하다는'},
 '브릿락': {'맞아', '쓰다', '참을'},
 '차분히': {'답답한', '적당히'},
 '무서운': {'해괴'}}

In [42]:
df.loc['의지', :]

슬픔     0
분노     0
기쁨     1
긴장감    0
무서운    0
증오     0
차분히    0
섹시     0
브릿락    0
설렘     0
몽환     0
Name: 의지, dtype: int64

In [43]:
df.drop('의지', inplace=True)

In [19]:
text

'폐 그걸 왜 찍어야 한다고 생각하는 거지 박상우가 눈매를 좁히며 김윤찬을 응시했다 그게 폐가 안 좋으셔서 오신 것 아닌가요 김윤찬은 시선을 살짝 피하며 어깨를 으쓱여 보였다 대통령님은 가벼운 협심증 증세를 보일 뿐이야 그래서 자네한테 검사 준비를 하라고 한 거고 아 죄송합니다 제가 착각을 했나 봅니다 김윤찬은 민망하다는 표정으로 뒷머리를 긁적거렸다 김윤찬 선생 대답은 그게 아니지 왜 폐 를 찍어야 한다고 했는지 제대로 된 설명이 필요할 것 같은데 말이야 그 제가 잘못 판단한 것일 수 있습니다 잘못 생각한 건지 아닌지는 내가 판단하겠네 대통령님의 폐에 문제가 있다고 생각한 근거는 뭔가 아 그게 박상우의 거듭된 물음에도 김윤찬은 쉬이 말을 잇지 못했다 지금 이렇게 자네와 대화할 시간은 없네만 박상우가 시계를 톡톡 건드리며 말하고 나서야 김윤찬이 입을 열었다 잘은 모르겠지만 대통령님의 목소리가 쉬어 있다고 느꼈습니다 목소리를 폐와 연관시키는 근거는 폐와 브롤커스 기관지 사이를 지나가는 신경으로 인해 목소리에 변화가 생겼다고 생각합니다 신경이라 뭐가 신경을 건드렸다고 생각하지 아직 검사를 진행하지 않아 장담할 수는 없지만 암세포일 가능성도 있습니다 암세포라 목소리가 쉬는 경우의 수만 해도 수십 수백 가지가 넘어 단적으로 래링쓰 후두 에 가벼운 인플라매토리 염증 만 생기도 목소리는 쉽게 변하지 암세포라고 말하고 싶으면 좀 더 날 설득해야 할 거야 그렇다면 제가 좀 더 교수님을 설득해도 되겠습니까 물론이야 내가 바라던 바니까 박상우는 당돌하지만 침착한 어조로 자신의 생각을 밝히는 김윤찬이 나름 귀여웠다 때문에 그에게 좀 더 기회를 주고 싶었다 대통령님께서 진료실 안으로 들어오실 때 다리를 절며 들어오셨습니다 다리를 절었다 네 그렇습니다 확실하지 않지만 대통령님은 최근에 프랙처 골절 혹은 스프레인 골염좌 이 있었던 같습니다 계속해 봐 박상우는 곰곰이 생각하며 말했다 폐 선암이 생기면 갑작스러운 통증을 수반하거나 골절이 생기기 쉽기 때문입니다 골 전이가 가장 잘되는 암은 

In [44]:
word_counts = {}
for i in word_li:
    if i in word_counts:
        word_counts[i] += 1
    else:
        word_counts[i] = 1

In [45]:
sorted(word_counts.items(), key = lambda item: item[1], reverse=True)

[('건일', 21),
 ('단명', 16),
 ('했다', 14),
 ('장유나', 13),
 ('여자', 12),
 ('변호사', 11),
 ('류지훈', 11),
 ('박연숙', 9),
 ('박연', 9),
 ('윤해성', 9),
 ('오는', 8),
 ('하고', 7),
 ('한테', 7),
 ('에서', 7),
 ('숙이', 7),
 ('사람', 7),
 ('구속', 7),
 ('그건', 6),
 ('엄마', 6),
 ('이다', 6),
 ('라고', 6),
 ('자리', 5),
 ('웃었다', 5),
 ('모양', 5),
 ('재산', 5),
 ('그런', 5),
 ('유방', 5),
 ('부인', 5),
 ('어떻게', 5),
 ('근데', 4),
 ('그래도', 4),
 ('우리', 4),
 ('에게', 4),
 ('수도', 4),
 ('없는', 4),
 ('오가', 4),
 ('누구', 4),
 ('보다', 4),
 ('욕심', 4),
 ('조금', 4),
 ('그룹', 4),
 ('연락', 4),
 ('이혼', 3),
 ('물론', 3),
 ('이에요', 3),
 ('그렇게', 3),
 ('회장', 3),
 ('있어요', 3),
 ('예요', 3),
 ('그저', 3),
 ('아니고', 3),
 ('해도', 3),
 ('생각', 3),
 ('지금', 3),
 ('있을', 3),
 ('자신', 3),
 ('어떤', 3),
 ('약물', 3),
 ('같은', 3),
 ('바로', 3),
 ('아들', 3),
 ('물려', 3),
 ('하려', 3),
 ('해서', 3),
 ('케이', 3),
 ('아니', 3),
 ('이란', 3),
 ('거야', 3),
 ('검사', 3),
 ('담당', 3),
 ('전화', 3),
 ('무슨', 3),
 ('일이', 3),
 ('고개', 3),
 ('설마', 2),
 ('형님', 2),
 ('아니에요', 2),
 ('형수', 2),
 ('이유', 2),
 ('때문', 2),
 ('빙그레', 2),
 ('혼인신

In [46]:
'구속' in df.index

True

In [47]:
df.columns[df.loc['구속'] == 1]

Index(['분노'], dtype='object')

In [52]:
mood_dic = {'슬픔': 0,
'분노': 1,
'기쁨': 2,
'긴장감': 3,
'무서운': 4,
'증오': 5,
'차분히': 6,
'섹시': 7,
'브릿락': 8,
'설렘': 9,
'몽환': 10}

In [53]:
def append_word(word, mood_li):
    columns_num = [0,0,0,0,0,0,0,0,0,0,0]
    for mood in mood_li:
        num = mood_dic[mood]
        columns_num[num] += 1
    columns_num = tuple(columns_num)
    tem_df = pd.DataFrame(data=[columns_num], index=[word], columns=['슬픔', '분노', '기쁨', '긴장감', '무서운', '증오', '차분히', '섹시', '브릿락', '설렘', '몽환'])
    if word not in df.index:
        return df.append(tem_df)
    else:
        return df

In [54]:
df = append_word('구속', ['긴장감', '섹시', '증오'])

In [55]:
df.tail(3)

,슬픔,분노,기쁨,긴장감,무서운,증오,차분히,섹시,브릿락,설렘,몽환
씁쓸한,1,0,0,0,0,0,0,0,0,0,0
급히,0,0,0,1,0,0,0,0,0,0,0
구속,0,0,0,1,0,1,0,1,0,0,0


In [56]:
df.to_excel('data_files/감성사전_편집.xlsx')

In [48]:
df.drop('구속', inplace=True)